In [2]:
pip install h3

Note: you may need to restart the kernel to use updated packages.


In [3]:
import h3

In [4]:
print(h3.__version__)

4.2.2


In [5]:
pip install h3 geopandas matplotlib

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


In [7]:
import folium
import h3

base_cell = '8001fffffffffff'  # Resolution 0 pentagon
children = h3.cell_to_children(base_cell, res=1)

# Create a map centered at the center of the base hexagon
base_center = h3.cell_to_latlng(base_cell)
GeoSpatialMap = folium.Map(location=[base_center[0], base_center[1]], zoom_start=9)

# Function to get hexagon boundaries
def get_hexagon_bounds(h3_address):
    boundaries = h3.cell_to_boundary(h3_address)
    # Folium expects coordinates in [lat, lon] format
    return [[lat, lng] for lat, lng in boundaries]

# Add base hexagon
folium.Polygon(
    locations=get_hexagon_bounds(base_cell),
    color='red',
    fill=True,
    weight=2,
    popup=f'Base: {base_cell}'
).add_to(GeoSpatialMap)

# Add children hexagons
for child in children:
    folium.Polygon(
        locations=get_hexagon_bounds(child),
        color='blue',
        fill=True,
        weight=1,
        popup=f'Child: {child}'
    ).add_to(GeoSpatialMap)

GeoSpatialMap

In [8]:
sydney_cell = h3.latlng_to_cell(-33.868820, 151.209290,9)# Sydney coordinates

#Traverse heirarchy upwards
parent =h3.cell_to_parent(sydney_cell,res=8)
print(f"Parent at res 8:{parent}")

#Traverse heirarchy downwards
children =h3.cell_to_children(parent, res=9)
print(f"Contains {len(children)} children")

#Create a new map centered on Sydney
sydney_map = folium.Map(location=[-33.868820,151.209290], zoom_starts=15)

# Add the parent hexagon (resolution 8)
folium.Polygon(
    locations=get_hexagon_bounds(parent),
    color='red',
    fill=True,
    weight=2,
    popup=f'Parent: {parent}'
).add_to(sydney_map)

# Add all children hexagons (resolution 9)
for child_cell in children:
    color = 'yellow' if child_cell == sydney_cell else 'blue'
    folium.Polygon(
        locations=get_hexagon_bounds(child_cell),
        color=color,
        fill=True,
        weight=1,
        popup=f'Child: {child_cell}'
    ).add_to(sydney_map)

sydney_map



Parent at res 8:88be0e35cbfffff
Contains 7 children


In [9]:
!pip install tensorflow

In [10]:
from tensorflow.keras.layers import LSTM


In [11]:
from tensorflow.keras.layers import Conv1D


In [12]:
from tensorflow.keras.layers import Dense


In [13]:
import h3
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Conv1D, Dense

In [14]:
# TRAFFIC PREDICTION CODE
#Create sample GPS data around Sydney
center_lat, center_lng = -33.868820, 151.209290
num_rides = 1000
np.random.seed(42)  # For reproducibility

In [15]:
#Generate random coordinates around Sydney
lats = np.random.normal(center_lat, 0.02, num_rides)
lngs = np.random.normal(center_lng, 0.02, num_rides)

In [16]:
# Generate timestamps for the past week
start_time = datetime.now() - timedelta(days=7)
timestamps = [start_time + timedelta(minutes=random.randint(0, 10080)) for _ in range(num_rides)]
timestamp_strs = [ts.strftime('%Y-%m-%d %H:%M:%S') for ts in timestamps]


In [17]:
# Generate random speed data
speeds = np.random.uniform(5, 60, num_rides)  # Speed in km/h


In [18]:
# Create DataFrame
gps_data = pd.DataFrame({
    'lat': lats,
    'lng': lngs,
    'timestamp': timestamp_strs,
    'speed': speeds
})

# Convert coordinates to H3 indexes (resolution 10)
gps_data["h3"] = gps_data.apply(
    lambda row: h3.latlng_to_cell(row["lat"], row["lng"], 10), axis=1
)

# Convert timestamp string to datetime objects
gps_data["timestamp"] = pd.to_datetime(gps_data["timestamp"])

In [19]:
# Aggregate speed and count per cell per 5-minute interval
agg_data = gps_data.groupby(["h3", pd.Grouper(key="timestamp", freq="5min")]).agg(
    avg_speed=("speed", "mean"),
    vehicle_count=("h3", "count")
).reset_index()

In [20]:
# Example: Use a cell from our existing dataset
sample_cell = gps_data["h3"].iloc[0]
neighbors = h3.grid_disk(sample_cell, 2)

def get_kring_features(cell, k=2):
    neighbors = h3.grid_disk(cell, k)
    return {f"neighbor_{i}": neighbor for i, neighbor in enumerate(neighbors)}

In [21]:
# Placeholder function for feature extraction
def fetch_features(neighbors, agg_data):
    # In a real implementation, this would fetch historical data for the neighbors
    # This is just a simplified example that returns random data
    return np.random.rand(1, 6, len(neighbors))  # 1 sample, 6 timesteps, features per neighbor

In [22]:
# Define a skeleton model architecture
def create_model(input_shape):
    model = tf.keras.Sequential([
        LSTM(64, return_sequences=True, input_shape=input_shape),
        LSTM(32),
        Dense(16, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [23]:
# Prediction function (would use a trained model in practice)
def predict_congestion(cell, model, agg_data):
    # Fetch neighbor cells
    neighbors = h3.grid_disk(cell, k=2)
    # Get historical data for neighbors
    features = fetch_features(neighbors, agg_data)
    # Predict
    return model.predict(features)[0][0]

In [24]:
# Create a skeleton model (not trained)
input_shape = (6, 19)  # 6 time steps, 19 features (for k=2 neighbors)
model = create_model(input_shape)

# Print information about what would happen in a real prediction
print(f"Sample cell: {sample_cell}")
print(f"Number of neighboring cells (k=2): {len(neighbors)}")
print("Model summary:")
model.summary()

# In practice, you would train the model before using it for predictions
# This would just show what a prediction call would look like:
congestion_prob = predict_congestion(sample_cell, model, agg_data)
print(f"Congestion probability: {congestion_prob:.2%}")

Sample cell: 8abe0e351a97fff
Number of neighboring cells (k=2): 19
Model summary:


C:\Users\rinit\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 6, 64)          │        21,504 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 34,465 (134.63 KB)

 Trainable params: 34,465 (134.63 KB)

 Non-trainable params: 0 (0.00 B)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 502ms/step
Congestion probability: 50.30%


In [25]:
import h3
import folium
import pandas as pd
import numpy as np
from folium.plugins import MarkerCluster

In [26]:
#DISASTER RESPONSE AND ENVIRONMENTAL MONITORING
#Create sample building dataset
np.random.seed(42)
num_buildings = 50

In [27]:
# Create buildings around Sydney
center_lat, center_lng = -33.868820, 151.209290
building_types = ['residential', 'commercial', 'hospital', 'school', 'government']
building_weights = [0.6, 0.2, 0.1, 0.07, 0.03]  # Probability weights

In [28]:
# Generate building data
buildings_df = pd.DataFrame({
    'lat': np.random.normal(center_lat, 0.005, num_buildings),
    'lng': np.random.normal(center_lng, 0.005, num_buildings),
    'type': np.random.choice(building_types, size=num_buildings, p=building_weights),
    'capacity': np.random.randint(10, 1000, num_buildings)
})

In [29]:
# Add H3 index at resolution 10
buildings_df['h3_index'] = buildings_df.apply(
    lambda row: h3.latlng_to_cell(row['lat'], row['lng'], 10),
    axis=1
)

In [30]:
# Create some flood cells (let's use some cells where buildings are located)
# Taking a few cells where buildings are located to simulate a flood zone
flood_cells = set(buildings_df['h3_index'].sample(10))

In [31]:
# Create a map centered at the average of our coordinates
center_lat = buildings_df['lat'].mean()
center_lng = buildings_df['lng'].mean()
flood_map = folium.Map(location=[center_lat, center_lng], zoom_start=16)

In [32]:
# Function to get hexagon boundaries for folium
def get_hexagon_bounds(h3_address):
    boundaries = h3.cell_to_boundary(h3_address)
    # Folium expects coordinates in [lat, lng] format
    return [[lat, lng] for lat, lng in boundaries]

In [33]:
# Add flood zone cells
for cell in flood_cells:
    folium.Polygon(
        locations=get_hexagon_bounds(cell),
        color='blue',
        fill=True,
        fill_opacity=0.4,
        weight=2,
        popup=f'Flood Cell: {cell}'
    ).add_to(flood_map)


In [34]:
# Add building markers
for idx, row in buildings_df.iterrows():
    # Set color based on if building is affected
    if row['h3_index'] in flood_cells:
        color = 'red'
        icon = 'warning' if row['type'] in ['hospital', 'school'] else 'info-sign'
        prefix = 'glyphicon'
    else:
        color = 'green'
        icon = 'home'
        prefix = 'glyphicon'

In [35]:
 # Create marker with popup showing building details
folium.Marker(
        location=[row['lat'], row['lng']],
        popup=f"Building Type: {row['type']}<br>Capacity: {row['capacity']}",
        tooltip=f"{row['type']} (Capacity: {row['capacity']})",
        icon=folium.Icon(color=color, icon=icon, prefix=prefix)
    ).add_to(flood_map)

In [36]:
# Add a legend as an HTML element
legend_html = '''
<div style="position: fixed; 
            bottom: 50px; left: 50px; width: 200px; height: 120px; 
            border:2px solid grey; z-index:9999; font-size:14px;
            background-color:white; padding: 10px;
            border-radius: 5px;">
      <b>Flood Impact Analysis</b> <br>
      <i class="glyphicon glyphicon-stop" style="color:blue"></i> Flood Zone <br>
      <i class="glyphicon glyphicon-home" style="color:green"></i> Safe Buildings <br>
      <i class="glyphicon glyphicon-info-sign" style="color:red"></i> Affected Buildings <br>
      <i class="glyphicon glyphicon-warning-sign" style="color:red"></i> Critical Facilities <br>
</div>
'''
flood_map.get_root().html.add_child(folium.Element(legend_html))

# Display the map
flood_map